## 0.Importar librerías iniciales e instancia de modelo de chat

In [1]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate, SystemMessagePromptTemplate,ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.agents import load_tools,initialize_agent,AgentType,create_react_agent,AgentExecutor
f = open('/home/iabd/clave_gpt.txt')
api_key = f.read().strip()
llm = ChatOpenAI(openai_api_key=api_key,temperature=0) #Recomendable temperatura a 0 para que el LLM no sea muy creativo, vamos a tener muchas herramientas a nuestra disposición y queremos que sea más determinista

## 1.Conectamos a la BBDD SQL

In [3]:
import mysql.connector #pip install mysql-connector-python

In [10]:
#f = open('../password_sql.txt')
#pass_sql = f.read()
# Configuración de la conexión a la base de datos
config = {
    'user': 'admin',       
    #'password': pass_sql, 
    'password': 'password', 
    'host': '127.0.0.1',         
    'database': 'world'          
}

In [11]:
# Conectar a la base de datos
conn = mysql.connector.connect(**config)
cursor = conn.cursor()

# 2. Ejecutamos consulta manualmente (sin agentes Langchain)

In [13]:
# Definir la consulta manualmente: tengo una base de datos mysql en mi computadora local denominada "world" y una tabla "Country" 
#sobre la que quiero hacer la suma de la población en la columna "Population" para el continente Asia (columna "Continent")
query = """
    SELECT SUM(Population)
    FROM country
    WHERE Continent = 'Asia';
    """

# Ejecutar la consulta
cursor.execute(query)
result = cursor.fetchone()

In [14]:
suma_poblacion = result[0] if result[0] is not None else 0
print(f"La suma de la población del continente Asia es: {suma_poblacion}")

La suma de la población del continente Asia es: 3705025700


## 3.Creamos el agente SQL 

In [15]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain.sql_database import SQLDatabase

In [16]:
# Crear una cadena de conexión a la base de datos MySQL
connection_string = f"mysql+mysqlconnector://{config['user']}:{config['password']}@{config['host']}/{config['database']}"

# Crear una instancia de la base de datos SQL
db = SQLDatabase.from_uri(connection_string)

In [17]:
agent = create_sql_agent(
    llm,
    db=db,
    verbose=True
)

In [18]:
agent.invoke("Dime la población total de Asia")



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: city, country, countrylanguageI should query the country table to get the population of Asia.
Action: sql_db_schema
Action Input: country
CREATE TABLE country (
	`Code` CHAR(3) NOT NULL DEFAULT '', 
	`Name` CHAR(52) NOT NULL DEFAULT '', 
	`Continent` ENUM('Asia','Europe','North America','Africa','Oceania','Antarctica','South America') NOT NULL DEFAULT 'Asia', 
	`Region` CHAR(26) NOT NULL DEFAULT '', 
	`SurfaceArea` FLOAT(10, 2) NOT NULL DEFAULT 0.00, 
	`IndepYear` SMALLINT(6), 
	`Population` INTEGER(11) NOT NULL DEFAULT 0, 
	`LifeExpectancy` FLOAT(3, 1), 
	`GNP` FLOAT(10, 2), 
	`GNPOld` FLOAT(10, 2), 
	`LocalName` CHAR(45) NOT NULL DEFAULT '', 
	`GovernmentForm` CHAR(45) NOT NULL DEFAULT '', 
	`HeadOfState` CHAR(60), 
	`Capital` INTEGER(11), 
	`Code2` CHAR(2) NOT NULL DEFAULT '', 
	PRIMARY KEY (`Code`)
)ENGINE=InnoDB COLLATE latin1_swedish_ci DEFAULT CHARSET=latin1

/*
3 rows from country table:
Code	

{'input': 'Dime la población total de Asia',
 'output': 'The total population of Asia is 3,879,000,000'}

In [19]:
result = agent.invoke("Dime el promedio de la esperanza de vida por cada una de las regiones ordenadas de mayor a menor")



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: city, country, countrylanguageI should query the schema of the country table to see if it contains information about regions and life expectancy.
Action: sql_db_schema
Action Input: country
CREATE TABLE country (
	`Code` CHAR(3) NOT NULL DEFAULT '', 
	`Name` CHAR(52) NOT NULL DEFAULT '', 
	`Continent` ENUM('Asia','Europe','North America','Africa','Oceania','Antarctica','South America') NOT NULL DEFAULT 'Asia', 
	`Region` CHAR(26) NOT NULL DEFAULT '', 
	`SurfaceArea` FLOAT(10, 2) NOT NULL DEFAULT 0.00, 
	`IndepYear` SMALLINT(6), 
	`Population` INTEGER(11) NOT NULL DEFAULT 0, 
	`LifeExpectancy` FLOAT(3, 1), 
	`GNP` FLOAT(10, 2), 
	`GNPOld` FLOAT(10, 2), 
	`LocalName` CHAR(45) NOT NULL DEFAULT '', 
	`GovernmentForm` CHAR(45) NOT NULL DEFAULT '', 
	`HeadOfState` CHAR(60), 
	`Capital` INTEGER(11), 
	`Code2` CHAR(2) NOT NULL DEFAULT '', 
	PRIMARY KEY (`Code`)
)ENGINE=InnoDB COLLATE latin1_swedish_ci DEFAULT

In [20]:
# Mostrar el resultado
print(result["output"])

The average life expectancy for each region, ordered from highest to lowest, is as follows:
1. Australia and New Zealand: 78.8
2. Nordic Countries: 78.33333
3. Western Europe: 78.25556
4. British Islands: 77.25
5. Southern Europe: 76.52857
6. North America: 75.82
7. Eastern Asia: 75.25
8. Caribbean: 73.05833
9. Central America: 71.025
10. South America: 70.94615


In [21]:
# Para utilizar few-shoots para las consultas SQL: https://python.langchain.com/v0.1/docs/use_cases/sql/agents/